In [ ]:
!pip install torch 
import torch

print(torch.__version__)

In [ ]:
import numpy as np
import warnings
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from collections import Counter, deque
import pandas as pd
import plotly.figure_factory as ff

import os
import glob
from tqdm.notebook import tqdm
import io
import pickle
import torch

\connecting Google drive for data

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link1 = '1DK0U6_0v4N-v70_zUjei0B4pVOgKceuo' #download data
downloaded = drive.CreateFile({'id':link1}) 
downloaded.GetContentFile('merged_data.csv')  
df_train = pd.read_csv('merged_data.csv', header=None) # training data

In [ ]:
link2 = '1zh76HFJ7j5ZqWg4Rqe1ClM7107jDaI0M' #download data
downloaded = drive.CreateFile({'id':link2}) 
downloaded.GetContentFile('merged_dev.csv')  
df_test = pd.read_csv('merged_dev.csv', header=None) # dev data

In [ ]:
!#unzip /content/out.zip -d /content/train/

Data checking

In [ ]:
df_test.head()


Build ANN

In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import glob

class Dataset(torch.utils.data.Dataset):
    def __init__(self,filenames):
      df = pd.read_csv(filenames)
      self.features = df.iloc[:,:-4]
      self.label = df.iloc[:,-3:]

    def __getitem__(self, index):
      features = self.features[index]
      label = self.label[index]
      return file#, labels

    def __len__(self):
      return len(self.features)




In [ ]:
train_path = '/content/train/merged_train.csv'
dev = '/content/dev/merged_dev.csv'

train_dataset = Dataset(train_path)
dev_dataset = Dataset(dev)


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=64, shuffle=True)

print(len(train_loader))
#print(len(dev_loader))

In [ ]:
import torch.nn as nn

class SLnetwork(nn.Module):
    def __init__(self):
        super(SLnetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.nn.Linear(356, 100)
        x = self.linear_relu_stack()
        x = self.Softmax(dim = 8)
        return x

In [ ]:
import torch.optim as optim

model = SLnetwork()
# Define the cost function
criterion = nn.CrossEntropyLoss()

# Define the optimizer, learning rate 
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
for epoch in range(2): 
      for i, (inputs, labels) in enumerate(train_loader, 0):
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels.squeeze(1))
        
        loss.backward()
        optimizer.step()
        cost = loss.item()
        if i % 100 == 0:
            print('Epoch:' + str(epoch) + ", Iteration: " + str(i) 
                  + ", training cost = " + str(cost))

In [ ]:
def calculate_accuracy(loader):
    total = 0
    correct = 0
  
    all_record = []
    all_preds = []
    all_label = []
    with torch.no_grad():
        for data in loader:
            record, label = data
            outputs = model(record)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.squeeze()).sum().item()
            
            all_record.append(record)
            all_preds.append(predicted.numpy())
            all_labels.append(labels)

    return 100 * correct / total, all_record, all_preds, all_labels

In [ ]:
train_accuracy, _ , _, _ = calculate_accuracy(train_loader)
test_accuracy, record, preds, labels = calculate_accuracy(test_loader)

print('Train accuracy: %f' % train_accuracy)
print('Test accuracy: %f' % test_accuracy)

record = np.concatenate(record, axis=0)
preds = np.concatenate(preds, axis=0)
labels = np.squeeze(np.concatenate(labels, axis=0))